In [1]:
from os import chdir

chdir("..")

from app.core.config import settings

# LLM

In [3]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama.chat_models import ChatOllama

chat_model_1b = ChatOllama(
    base_url=settings.OLLAMA_BASE_URL,
    model="llama3.2:1b",
    temperature=0,
)

chat_model_3b = ChatOllama(
    base_url=settings.OLLAMA_BASE_URL,
    model="llama3.2:3b",
    temperature=0,
)

embeddings = OllamaEmbeddings(base_url=settings.OLLAMA_BASE_URL, model=settings.OLLAMA_EMBEDDINGS_MODEL)

## Structured Output

In [7]:
from pydantic import BaseModel, field_validator

class Navigation(BaseModel):
    id: int
    reasoning: str

In [8]:
chat_model_1b_structured = chat_model_1b.with_structured_output(Navigation, method="json_schema")
chat_model_3b_structured = chat_model_3b.with_structured_output(Navigation, method="json_schema")

In [9]:
chat_model_1b_structured.invoke("Your ID is 1, reasoning is 'test case'"), chat_model_3b_structured.invoke("Your ID is 1, reasoning is 'test case'")

(Navigation(id=1, reasoning='test case'),
 Navigation(id=1, reasoning='test case'))

# Vector DB

In [12]:
from langchain_chroma import Chroma

PERSIST_DIRECTORY = "./data/vector_db/chroma/"
vectordb = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)

In [13]:
retriever = vectordb.as_retriever()

## Dataset

In [14]:
from json import load, dump

In [15]:
with open("./data/navigation/navigation_intents.json") as f:
    intents = load(f)

In [16]:
dataset = {}

for index, item in enumerate(intents, start=1):
    dataset[index] = item

In [18]:
len(dataset.keys()) == len(intents)

True

# RAG

In [23]:
from langchain_core.prompts import PromptTemplate


RAG_PROMPT = """
You are an navigation assistant. You are given a list of navigation options with their ID.
Your task is to identify the option that is most similar to the user {query}
Query: {query}
Context: {context}
Schema:
{{

    "id": ID,
    "reasoning": <Reasoning for selecting this ID
}}
Output:
"""

template = PromptTemplate.from_template(RAG_PROMPT)

rag_chain_1b = template | chat_model_1b_structured
rag_chain_3b = template | chat_model_3b_structured

## Graph

In [24]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
    question: str
    context: List[Document]
    answer: dict

In [25]:
def retrieve(state: State):
    retrieved_docs = vectordb.similarity_search(state["question"])
    return {"context": retrieved_docs}

In [35]:
def generate(state: State):
    context = ""
    for doc in state["context"]:
        context += f"{doc.metadata['id']} - {doc.page_content} | "

    # print(f"Context: {context}")
    
    response1 = rag_chain_1b.invoke({"query": state["question"], "context": context})
    response2 = rag_chain_3b.invoke({"query": state["question"], "context": context})
    return {"answer": {
            "1b": response1,
            "3b": response2
        }
    }

In [36]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [37]:
QUERY = "user wants to manage assignments of products"

In [38]:
response = graph.invoke({"question": QUERY})
print(response["answer"])

{'1b': Navigation(id=36, reasoning='Takes the user to the assignment screen to manage assignments of products'), '3b': Navigation(id=36, reasoning='Takes the user to the assignment screen to manage assignments of products or workers.')}


## Testing

In [31]:
with open("./data/navigation/navigation_test_queries.json") as f:
    tests = load(f)

In [34]:
tests[0]

{'id': 1,
 'description': 'Directs the user to the login page to authenticate and access the application.',
 'test_queries': ['Open login page',
  'Go to login screen',
  'Navigate to sign-in page',
  'Take me to login',
  'Show login view',
  'Direct me to authentication page',
  'Access login section',
  'Open sign-in screen',
  'Navigate to login area',
  'Go to user login']}

In [39]:
!pip install tqdm

In [49]:
from tqdm import tqdm

results = {}

for test in tqdm(tests):
    id = test["id"]
    description = test["description"]

    print(f"ID: {id}")
    
    accuracy_1b, accuracy_3b = 0, 0
    
    for query in tqdm(test["test_queries"]):
        response = graph.invoke({"question": query})
        answer = response["answer"]
        try:
            id_1b, id_3b = answer["1b"].id, answer["3b"].id
            accuracy_1b += id_1b == id
            accuracy_3b += id_3b == id

            results[id] = {
                "id": id,
                "description": description,
                "accuracy_1b": accuracy_1b/10 * 100,
                "accuracy_3b": accuracy_3b/10 * 100
            }
        except Exception as e:
            print(e)


  0%|                                                              | 0/41 [00:00<?, ?it/s]

ID: 1



  2%|█▎                                                    | 1/41 [01:20<53:42, 80.57s/it]

ID: 2



  5%|██▋                                                   | 2/41 [02:49<55:31, 85.42s/it]

...


 95%|██████████████████████████████████████████████████▍  | 39/41 [55:38<02:47, 83.80s/it]

ID: 40



 98%|███████████████████████████████████████████████████▋ | 40/41 [57:04<01:24, 84.52s/it]

ID: 41



100%|█████████████████████████████████████████████████████| 41/41 [58:29<00:00, 85.60s/it]


In [50]:
results

{1: {'id': 1,
  'description': 'Directs the user to the login page to authenticate and access the application.',
  'accuracy_1b': 0.0,
  'accuracy_3b': 90.0},
 2: {'id': 2,
  'description': 'Navigates to the main dashboard, providing an overview of key application metrics and access points.',
  'accuracy_1b': 70.0,
  'accuracy_3b': 80.0},
 3: {'id': 3,
  'description': 'Takes the user to the application dashboard to view detailed application-specific data and controls.',
  'accuracy_1b': 40.0,
  'accuracy_3b': 50.0},
 4: {'id': 4,
  'description': 'Directs to the tab list page displaying worker or event information in a tabular format.',
  'accuracy_1b': 40.0,
  'accuracy_3b': 60.0},
 5: {'id': 5,
  'description': 'Navigates to a detailed view of a specific worker or event, including relevant metadata.',
  'accuracy_1b': 50.0,
  'accuracy_3b': 60.0},
 6: {'id': 6,
  'description': 'Takes the user to the event resolution page to take actions on specific events.',
  'accuracy_1b': 30.0,


### Compare results

In [53]:
score_1b, score_3b = 0,0

for i in results.values():
    if i["accuracy_1b"] > i["accuracy_3b"]:
        score_1b += 1
    else:
        score_3b += 1

In [60]:
print(f"Accuracy Score:\nLlama 3.1 1B {score_1b}/{len(intents)}\nLlama 3.2 3B {score_3b}/{len(intents)}")

Accuracy Score:
Llama 3.1 1B 9/41
Llama 3.2 3B 32/41


### Validation

1. On average 3B is better than 1B
2. A secondary validation to check 